In [6]:
import pandas as pd
import sys
sys.path.append('../src')
from cellopype import Cell, Pype

In [7]:
pp = Pype()
dfA = pd.DataFrame([1, 2, 3], columns=["value"])
dfB = pd.DataFrame([10, 20, 30], columns=["value"])

pp.a = Cell(recalc=lambda: dfA)
pp.b = Cell(recalc=lambda: dfB)
pp.c = Cell(recalc=pd.DataFrame.add, sources=[pp.a, pp.b])

In [8]:
# cells in a Pype get their name as an attribute on the cell
pp.a.name

'a'

In [9]:
pp.a._value

In [10]:
# recalculate and dump all cell values in pp 
lits = pp.dump_values()

In [11]:
# lits is a list of dicts
# NOTE: dataframe and series values are COPIED by dump_values for pseudo-immutability
lits

[{'name': 'a',
  'value':    value
  0      1
  1      2
  2      3},
 {'name': 'b',
  'value':    value
  0     10
  1     20
  2     30},
 {'name': 'c',
  'value':    value
  0     11
  1     22
  2     33}]

In [12]:
pp.a._value.loc[0,'value']=99

In [13]:
# show the local cell value in pp has changed
pp.a._value

,value
0,99
1,2
2,3


In [14]:
# restore values from the dump
pp.load_values(lits)
pp.a._value

,value
0,1
1,2
2,3


In [14]:
# if you want to dump & restore the entire pype, including recalc logic & handlers:
import dill  # not pickle, because lambdas

with open('pype_p.dill', 'wb') as file:
    dill.dump(pp, file)
with open('pype_p.dill', 'rb') as inp:
    pp2 = dill.load(inp)

In [16]:
# the pp2 pype is now a full & independent copy of pp
pp2.c.sources

In [18]:
pp2.c.recalc_handler(pp2.a.value, pp2.b.value)

,value
0,11
1,22
2,33


In [4]:
# new pype, with force recalc for all cells in this pype (lazy off) 
# only available on init (=before adding cells)
ppp = Pype(force_unlazy=True)

In [15]:
ppp.force_unlazy

True